# LangGraph: Basic Multi-Agent App to replace a Content Marketing Team

## Why use LangGraph?
* To build Multi-Agent LLM Apps.
* LangGraph can coordinate multiple agents.
* The LLM Apps of the future: Agentic Behavior.

In [2]:
import functools
import operator
import requests
import os
from dotenv import load_dotenv, find_dotenv
from bs4 import BeautifulSoup
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage, BaseMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI

from typing import TypedDict, Annotated, Sequence

from langchain_community.tools.tavily_search import TavilySearchResults


In [3]:
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

## Select the LLM

In [4]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

## Define the tools to use

## Define the tools to use
* Online searching with Tavily
* HTML parsing with BeautifulSoup
* return_direct=False
    * Meaning: this tool will be returning results privately to the agent, not publicly to the app.

In [5]:
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str)->str:
    """Used to process content found on the internet"""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()


tools = [TavilySearchResults(max_results=1), process_search_tool]

for demo and costs purposes we have limited the search results to just 1. This indeed will limit the quality of the online search since our online search agent will only study one search result

## Define the function to create new agents
* Arguments: llm, tool list, system prompt.
* OpenAI tools agent.
* Output: AgentExecutor

In [7]:
def create_new_agent(llm: ChatOpenAI,
                     tools: list,
                     system_prompt: str) -> AgentExecutor:
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        MessagesPlaceholder(variable_name='messages'),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ])
    agent = create_openai_tools_agent(llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

This previous function is designed to set up and return an agent that can perform tasks using OpenAI's language model and a set of tools. Here’s a breakdown of what each part does:

1. **Function Definition (`create_new_agent`)**:
   - It takes four parameters:
     - `llm`: an instance of `ChatOpenAI`, which refers to the language model from OpenAI.
     - `tools`: a list of tools that the agent can use.
     - `system_prompt`: a string that provides an initial message or command for the agent.
   - The function is expected to return an instance of `AgentExecutor`.

2. **Creating a Prompt**:
   - `prompt = ChatPromptTemplate.from_messages([...])`: This line constructs a prompt for the agent using a template. The prompt includes:
     - A system message (`system_prompt`).
     - Two placeholders for additional dynamic content, labeled `"messages"` and `"agent_scratchpad"`. These are used to insert messages that the agent receives during its operation and notes or data that the agent might need to keep track of, respectively.

3. **Creating an Agent**:
   - `agent = create_openai_tools_agent(llm, tools, prompt)`: This line initializes the agent using the language model (`llm`), the list of tools (`tools`), and the prompt created earlier. The function (`create_openai_tools_agent`) is designed to configure the agent with the necessary components to function properly.

4. **Creating an Executor**:
   - `executor = AgentExecutor(agent=agent, tools=tools)`: This creates an `AgentExecutor` object, which manages the execution of the agent's tasks. It is configured with the agent itself and the tools the agent can use.

5. **Return the Executor**:
   - `return executor`: The function finishes by returning the `executor` object, which is now ready to handle tasks according to the `system_prompt` and interact using the tools provided.

In simple terms, this function sets up an agent with specific tools and a starting instruction, ready to perform tasks as instructed by further interactions.

## Define the function to create a node in the multi-agent network
* Nodes are like the way we locate each agent in the network.

In [8]:
def agent_node(state, agent, name):
     result = agent.invoke(state)
     return {"messages": [HumanMessage(content=result["output"], name=name)]}

## Create a list with the names of the AI Agents we will create to replace our content marketing team

In [9]:
content_marketing_team = ["online_researcher", "blog_manager", "social_media_manager"]

## Define the system prompt
* The system_prompt defines the role of the content marketing manager. The content marketing manager decides when an agent should intervene in the network cycle.
* Each agent delivers a report to the content marketing manager when her task is finished, and then the content marketing manager decides what to do next, if another agent takes charge or if the cycle finishes.

In [10]:
system_prompt = (
     "As a content marketing manager, your role is to oversee the insight between these"
     " workers: {content_marketing_team}. Based on the user's request,"
     " determine which worker should take the next action. Each worker is responsible for"
     " executing a specific task and reporting back thier findings and progress."
     " Once all tasks are completed, indicate 'FINISH'."
 )

## Create the options list
* This list presents all the options the content marketing manager can choose from: the list of agent names and the "FINISH" option to finish the cycle.

In [11]:
options = ["FINISH"] + content_marketing_team

## Define the routeSchema function
* This function determines how to select the next agent in charge in the next cycle stage, the next role.
* As you can see, in the properties it includes the options list.

In [12]:
function_def = {
     "name": "route",
     "description": "Select the next role.",
     "parameters": {
         "title": "routeSchema",
         "type": "object",
         "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}]}},
         "required": ["next"]
     }
 }

The previous function described in the JSON object is named `route`, and its primary purpose is to select the next role based on a specific schema called `routeSchema`. Let us break down the various components to clarify what each part does:

1. **Name**: The function is called `route`. The name is self explanatory: the function helps in determining a direction or making a decision in a sequence of actions.

2. **Description**: The description "Select the next role" is also self explanatory: the function's main task is to determine and select what the next role should be in our workflow.

3. **Parameters**: This function takes one parameter defined by the `routeSchema`. Here's how the schema is structured:
   
   - **title**: The title of the parameter is `routeSchema`.
   - **type**: It's of type `object`, meaning this parameter should be a structured object with properties defined under it.
   - **properties**:
     - The key property is `next`. It has the title "Next", and its value can be one of several options. These options are referenced as `options`, which means that `next` can take on any value from the options list.
   - **required**: The property `next` is required, meaning that for the function to operate, the `next` property must be provided in the object passed to the function.

## Define the content marketing manager's prompt
* Includes the system prompt.
* Asks what option to take next.

In [21]:
prompt = ChatPromptTemplate.from_messages([
     ("system", system_prompt),
     MessagesPlaceholder(variable_name="messages"),
     ("system",
      "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
 ]).partial(options=str(options), content_marketing_team=", ".join(content_marketing_team))

The previous code is be constructing a prompt template for our chat-based application. Let's break down each part of the code for clarity:

1. **Creating a Prompt Template**:
   - `ChatPromptTemplate.from_messages([])`: This is a method used to create a new chat prompt template. It takes a list of messages as an argument, which defines the structure of the conversation that will be presented to the user.

2. **Defining the Conversation**:
   - Inside the list passed to `from_messages`, there are tuples and placeholders that represent different parts of a conversation:
     - `("system", system_prompt)`: This tuple represents a message from the "system" using the variable `system_prompt`.
     - `MessagesPlaceholder(variable_name="messages")`: This is a placeholder for user messages that will be dynamically filled based on the actual conversation. The placeholder is labeled with a variable name `"messages"`, indicating where messages from the user or other participants should be inserted in the template.
     - `("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}")`: System message that prompts the user to decide the next step in the conversation, providing specific options that are dynamically determined.

3. **Dynamic Content**:
   - `.partial(options=str(options), content_marketing_team=", ".join(content_marketing_team))`: This part of the code modifies the template to include specific dynamic content:
     - `options=str(options)`: The placeholder `{options}` in the last system message is replaced with a string representation of the variable `options`, which contains different actions or choices the user can make.
     - `content_marketing_team=", ".join(content_marketing_team)`: the list `content_marketing_team` is converted to a comma-separated string.

## Define the content marketing manager's chain
* Use the content marketing manager's prompt
* Use the routerSchema function
* Parse the output as JSON

In [22]:
content_marketing_manager_chain = (prompt | llm.bind_functions(
    functions=[function_def], function_call="route") | JsonOutputFunctionsParser())

## Create the online_researcher agent
Remember, the create_new_agent function only requires 3 arguments:
* What LLM will the agent use.
* The list of tools the agent can use. In this exercise all angents share the same tool list, but they can have differente tool lists.
* What prompt defines the agent's role.

In [13]:
online_researcher_agent = create_new_agent(
     llm,
     tools,
     """Your primary role is to function as an intelligent online research assistant, adept at scouring 
     the internet for the latest and most relevant trending stories across various sectors like politics, technology, 
     health, culture, and global events. You possess the capability to access a wide range of online news sources, 
     blogs, and social media platforms to gather real-time information."""
 )

## Define the node where we can locate the previous agent
* Using functools, we basically clone the agent_node function and add online_researcher_agent as argument and giving the node the name of "online_researcher". See how functions.partial works below.

In [14]:
online_researcher_node = functools.partial(
     agent_node, agent=online_researcher_agent, name="online_researcher"
 )

#### What does functools.partial do?
The `functools` module is part of the Python standard library and **doesn't need to be installed separately.**

`functools` provides tools that help you modify and manage functions easily. Here’s a simplified explanation of some of its main features:

1. **`partial`**: Imagine you have a recipe that you always cook with a slight variation. Instead of rewriting the recipe each time, you can write down the common steps and just specify the variation when needed. `partial` helps you create a simplified version of a function by pre-setting some arguments.

2. **`lru_cache`**: This is like having a small notebook where you write down solutions to problems you've already solved. The next time you face the same problem, instead of solving it again, you just look at your notebook. This saves time, especially for complex problems.

3. **`reduce`**: Suppose you have a list of numbers and you want to combine them into one number, maybe by adding them up. `reduce` takes a function (like addition) and a list, applies the function to combine the items of the list into a single result.

4. **`total_ordering`**: If you have a class representing people with just their ages and you want to easily compare who is older or younger, you usually need to write several methods to compare them. With `total_ordering`, you just write one comparison method, and it figures out the rest for you.

5. **`singledispatch`**: This lets you write a general function that can behave differently depending on what type of thing it is dealing with. For example, you might have a function that needs to handle both numbers and strings differently with just one function definition.

6. **`cmp_to_key`**: Some tools in Python sort objects but need a special rule for how to rank them. `cmp_to_key` converts an old-style comparison function (which tells you which of two values is larger) into a key function that the sorting tool can use.

Overall, `functools` helps make functions more versatile and easier to manage in your programs.

## Repeat the previous two steps to create the other 2 agents with their corresponding nodes.

In [15]:
blog_manager_agent = create_new_agent(
     llm, tools,
     """You are a Blog Manager. The role of a Blog Manager encompasses several critical responsibilities aimed at transforming initial drafts into polished, SEO-optimized blog articles that engage and grow an audience. Starting with drafts provided by online researchers, the Blog Manager must thoroughly understand the content, ensuring it aligns with the blog's tone, target audience, and thematic goals. Key responsibilities include:

 1. Content Enhancement: Elevate the draft's quality by improving clarity, flow, and engagement. This involves refining the narrative, adding compelling headers, and ensuring the article is reader-friendly and informative.

 2. SEO Optimization: Implement best practices for search engine optimization. This includes keyword research and integration, optimizing meta descriptions, and ensuring URL structures and heading tags enhance visibility in search engine results.

 3. Compliance and Best Practices: Ensure the content adheres to legal and ethical standards, including copyright laws and truth in advertising. The Blog Manager must also keep up with evolving SEO strategies and blogging trends to maintain and enhance content effectiveness.

 4. Editorial Oversight: Work closely with writers and contributors to maintain a consistent voice and quality across all blog posts. This may also involve managing a content calendar, scheduling posts for optimal engagement, and coordinating with marketing teams to support promotional activities.

 5. Analytics and Feedback Integration: Regularly review performance metrics to understand audience engagement and preferences. Use this data to refine future content and optimize overall blog strategy.

 In summary, the Blog Manager plays a pivotal role in bridging initial research and the final publication by enhancing content quality, ensuring SEO compatibility, and aligning with the strategic objectives of the blog. This position requires a blend of creative, technical, and analytical skills to successfully manage and grow the blog's presence online.""")


blog_manager_node = functools.partial(
     agent_node, agent=blog_manager_agent, name="blog_manager")


social_media_manager_agent = create_new_agent(
     llm, tools,
     """You are a Social Media Manager. The role of a Social Media Manager, particularly for managing Twitter content, involves transforming research drafts into concise, engaging tweets that resonate with the audience and adhere to platform best practices. Upon receiving a draft from an online researcher, the Social Media Manager is tasked with several critical functions:

 1. Content Condensation: Distill the core message of the draft into a tweet, which typically allows for only 280 characters. This requires a sharp focus on brevity while maintaining the essence and impact of the message.

 2. Engagement Optimization: Craft tweets to maximize engagement. This includes the strategic use of compelling language, relevant hashtags, and timely topics that resonate with the target audience.

 3. Compliance and Best Practices: Ensure that the tweets follow Twitter’s guidelines and best practices, including the appropriate use of mentions, hashtags, and links. Also, adhere to ethical standards, avoiding misinformation and respecting copyright norms.

 In summary, the Social Media Manager's role is crucial in leveraging Twitter to disseminate information effectively, engage with followers, and build the brand’s presence online. This position combines creative communication skills with strategic planning and analysis to optimize social media impact.""")

social_media_manager_node = functools.partial(
     agent_node, agent=social_media_manager_agent, name="social_media_manager")

## Define Individual Agent Memory: the AgentState class 

In [16]:
class AgentState(TypedDict):
     messages: Annotated[Sequence[BaseMessage], operator.add]
     next: str

The previous code defines a Python class called `AgentState` using a feature called `TypedDict` from the `typing` module. Here’s what each part does in simple terms:

1. **`TypedDict`**: This is used to create a dictionary where you can specify the type of data each key should hold. It's a way to make sure your dictionaries are more predictable by ensuring each key has a specific type of value.

2. **`AgentState`**: This is the name of the dictionary type being defined. You can think of `AgentState` as a blueprint for creating dictionaries that always have the same structure.

3. **`messages`**: This is one of the keys in the `AgentState` dictionary. It is expected to hold a sequence (like a list or a tuple) of items, each of which is an instance of `BaseMessage`. `BaseMessage` is a Base class in LangChain for all types of messages in a conversation. It includes properties like `content`, `name`, and `additional_kwargs`. It also includes methods like `toDict()` and `_getType()`.

   - **`Annotated[Sequence[BaseMessage], operator.add]`**: Here, `Annotated` is used to add additional information to the type hint. This extra information is `operator.add`, suggesting that the values in the `messages` sequence might be combined using the addition operation, indicating that messages can be concatenated or aggregated. The `operator` module in Python provides a set of functions corresponding to the intrinsic operators of Python. For example, instead of using + for addition, operator.add(x, y) can be used. 

4. **`next`**: This is another key in the `AgentState` dictionary. It holds a value of type `str`, which means it's expected to be a string. This could be used to indicate the next action, state, or identifier in a process or workflow involving an "agent."

In summary, the `AgentState` class defines a template for creating dictionaries that keep track of an agent's messages and its next state or action. Each `AgentState` dictionary will have a list of `messages` that are instances of `BaseMessage` and a `next` value that is a string. The use of `Annotated` with `operator.add` might imply some special handling or processing of the messages that isn't standard in typical type hints.

## Let's now create the Workflow of the Agent Network, what in LangGraph is called a Graph or a Stateful Graph

In [17]:
workflow = StateGraph(AgentState)

## Let's now add the nodes of the Workflow (also called the nodes of the Graph)
* See that the action of the content_marketing_manager's node is the content_marketing_manager_chain

In [24]:
workflow.add_node("content_marketing_manager", action=content_marketing_manager_chain)
workflow.add_node("online_researcher", action=online_researcher_node)
workflow.add_node("blog_manager", action=blog_manager_node)
workflow.add_node("social_media_manager", action=social_media_manager_node)

a## And now we will add the connections among the nodes, what in LangGraph are called the "edges"


In [25]:
for member in content_marketing_team:
     workflow.add_edge(start_key=member, end_key="content_marketing_manager")

In [26]:
conditional_map = {k: k for k in content_marketing_team}

conditional_map['FINISH'] = END

workflow.add_conditional_edges(
    "content_marketing_manager", lambda x: x["next"], conditional_map)

workflow.set_entry_point("content_marketing_manager")

Here's a breakdown of each part of the code and what it does in simple terms:

1. **`conditional_map = {k: k for k in content_marketing_team}`**:
   - This line creates a dictionary named `conditional_map` where each key and its corresponding value are the same. The keys (and values) are taken from the list `content_marketing_team`.

2. **`conditional_map['FINISH'] = END`**:
   - Here, the dictionary `conditional_map` is updated to include a new key `'FINISH'` with a corresponding value `END`. See that we imported `END` from langgraph.graph in the top section of multiagent.py

3. **`workflow.add_conditional_edges("content_marketing_manager", lambda x: x["next"], conditional_map)`**:
   - This line adds conditional edges to a `workflow` object. The first argument `"content_marketing_manager"` is a node in the workflow.
   - The second argument is a lambda function `lambda x: x["next"]`. This function is used to determine the next step in the workflow based on the current state's `"next"` attribute. Essentially, it looks at the current context or state (`x`), retrieves what is specified by `"next"`, and uses this to decide the next action or node in the workflow.
   - The third argument, `conditional_map`, provides a mapping from possible values of `x["next"]` to what actually should happen next in the workflow. This enables dynamic decision-making within the workflow based on the current state.

4. **`workflow.set_entry_point("content_marketing_manager")`**:
   - Finally, this line sets the entry point of the workflow to `"content_marketing_manager"`. This means that `"content_marketing_manager"` is the starting point or initial state from which the workflow begins execution.

In essence, this code is configuring a workflow where execution starts at the `"content_marketing_manager"` node, uses a dictionary (`conditional_map`) to map potential next steps based on the current context, and dynamically determines the flow based on the current state's `"next"` value.

## Our final step, let's initialize the agent network (often called graph in LangGraph) and let's ask it to do some work for us

In [27]:
multiagent = workflow.compile()

for s in multiagent.stream(
     {
         "messages": [
             HumanMessage(
                 content="""Write me a report on Agentic Behavior. After the research on Agentic Behavior,pass the findings to the blog manager to generate the final blog article. Once done, pass it to the social media manager to write a tweet on the subject."""
             )
         ],
     },
     # Maximum number of steps to take in the graph
     {"recursion_limit": 150}
):
     if not "__end__" in s:
         print(s, end="\n\n-----------------\n\n")

{'content_marketing_manager': {'next': 'blog_manager'}}

-----------------

{'blog_manager': {'messages': [HumanMessage(content="I have retrieved information on Agentic Behavior from a research article. Let's proceed with creating a report on Agentic Behavior.\n# Report on Agentic Behavior\n\n## Introduction\nAgentic behavior refers to the actions and behaviors of individuals that are driven by a sense of agency and autonomy. This behavior is characterized by a proactive approach to decision-making and problem-solving, where individuals take control of their actions and outcomes.\n\n## Key Findings\nThe research on agentic behavior suggests that it has a positive effect on the individual level but may have null or negative effects on the dyadic level. This indicates that while agentic behavior can benefit individuals in terms of personal growth and achievement, it may not always translate positively in interpersonal relationships or group dynamics.\n\n## Implications\nUnderstanding age

## Take a look at LangSmith
* See that the project takes a lot of tokens.
* See that each agent is monitored.
* Click on each agent and see input and output.
* See how the online researcher agent is using Tavily for online searching.
* See the output of the content marketing manager in the last step ("FINISH").

## One important caveat: Multi-Agent Networks take time to deliver the final solution, so you will have to be patient.
* Multi-Agent Networks, also called LLM Apps with Agentic Behavior or Multi-Agent LLM Apps take time, but their results are often better than the ones got from Non-Agentic LLM Apps.